In [85]:
import folium
import pandas as pd
import numpy as np
import json
import statistics
import matplotlib.pyplot as plt

In [86]:
# Read population data from Excel file
crime_df = pd.read_csv('D:/PyCharm/pythonProject/143/Project/Data/Drug_Crime_20231127.csv',nrows=5000)

print(crime_df.columns)

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'PARKS_NM',
       'HADEVELOPT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lat_Lon'],
      dtype='object')


In [87]:
crime_data = {}

# Iterate over the DataFrame rows and populate the dictionary
for index, row in crime_df.iterrows():
    key = row['CMPLNT_NUM']
    values = {
        'Police_Depart': row['PD_CD'],
        'Coordinate': [row['Latitude'],row['Longitude']]
    }
    crime_data[key] = values


In [88]:
crime_data

{80488152: {'Police_Depart': 503,
  'Coordinate': [40.8162058439227, -73.8960011932583]},
 25436757: {'Police_Depart': 511,
  'Coordinate': [40.6713598203364, -73.8818110231735]},
 10354189: {'Police_Depart': 503,
  'Coordinate': [40.6799807384666, -73.7762339071953]},
 10049866: {'Police_Depart': 503,
  'Coordinate': [40.8222710411331, -73.911697780277]},
 26116195: {'Police_Depart': 511,
  'Coordinate': [40.8441566000203, -73.9006054489734]},
 50503943: {'Police_Depart': 503,
  'Coordinate': [40.6894642952604, -73.9240290899499]},
 23654852: {'Police_Depart': 511,
  'Coordinate': [40.81591307653, -73.9451493066481]},
 38821305: {'Police_Depart': 500,
  'Coordinate': [40.8307223277672, -73.8271260588639]},
 10016237: {'Police_Depart': 500,
  'Coordinate': [40.7072398161698, -73.7927267255908]},
 39325387: {'Police_Depart': 567,
  'Coordinate': [40.6836512406856, -74.0003043120176]},
 24522213: {'Police_Depart': 503,
  'Coordinate': [40.6713598203364, -73.8818110231735]},
 9965860: {'P

In [89]:
PD_values = [value['Police_Depart'] for value in crime_data.values() ]
                        
# Displaying the extracted values
max_PD = max(PD_values)
min_PD = min(PD_values)

# Create a color scale for population
color = ['yellow','green','blue', 'red','purple']

color_scale = folium.LinearColormap(colors=color, vmin=min_PD, vmax=max_PD)

In [90]:
# Load GeoJSON data for NYC community districts
with open('D:/PyCharm/pythonProject/143/Project/Data/Community Districts.geojson', 'r') as f:
    NYC_district = json.load(f)


In [91]:
# Create a map centered at New York City
Map_NYC = folium.Map(location=[40.7128, -74.0060], zoom_start=10, tiles='CartoDB positron')

# Add GeoJSON layer to the map
folium.GeoJson(
    NYC_district,
    name='nyc-districts & crime',

    style_function= lambda feature: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    
                                   
).add_to(Map_NYC)


In [92]:
for key, value in crime_data.items():

    coordinates = value.get('Coordinate')

    if isinstance(coordinates, list) and not any(np.isnan(coordinates)):  # Check if coordinates are a list and not NaN
        folium.CircleMarker(location=coordinates, radius=2, color=color_scale(value['Police_Depart']), fill=True, fill_color='red').add_to(Map_NYC)

        


In [93]:
# Add a marker with text at a specific location
folium.map.Marker(location=[40.76440432106826, -74.02372202390437], icon=folium.DivIcon(html=f"<div style='font-size: 15pt'>Manhattan</div>")).add_to(Map_NYC)
folium.map.Marker(location=[40.73536409760489, -73.85770225871684], icon=folium.DivIcon(html=f"<div style='font-size: 15pt'>Queens</div>")).add_to(Map_NYC)
folium.map.Marker(location=[40.64209332024007, -74.00113449211089], icon=folium.DivIcon(html=f"<div style='font-size: 15pt'>Brooklyn</div>")).add_to(Map_NYC)
folium.map.Marker(location=[40.60535622898584, -74.16354960167357], icon=folium.DivIcon(html=f"<div style='font-size: 15pt'>Staten Island</div>")).add_to(Map_NYC)
folium.map.Marker(location=[40.857210423454255, -73.90217959092037], icon=folium.DivIcon(html=f"<div style='font-size: 15pt'>Bronx</div>")).add_to(Map_NYC)

# Add legend for population color scale
color_scale.caption = 'Crime Category' 
color_scale.add_to(Map_NYC)


In [94]:
# Display the map
Map_NYC

In [ ]:
Map_NYC.save("Crime Map.html")